In [0]:
%sql
USE CATALOG catproject_catalog;
USE SCHEMA ecom_bronze;

In [0]:
%sql
CREATE TABLE ecom_bronze.customer_raw (
  customer_id         STRING,               -- Mã khách hàng duy nhất
  full_name           STRING,               -- Họ tên đầy đủ
  date_of_birth       DATE,                 -- Ngày sinh
  gender              STRING,               -- Giới tính
  email               STRING,               -- Địa chỉ email
  phone_number        STRING,               -- Số điện thoại
  address             STRING,               -- Địa chỉ thường trú
  city                STRING,               -- Thành phố
  country             STRING,               -- Quốc gia
  registration_date   TIMESTAMP,            -- Ngày đăng ký tài khoản
  loyalty_level       STRING,               -- Hạng thành viên: 'Silver', 'Gold', 'Platinum'
  status              STRING,               -- Trạng thái tài khoản: 'active', 'inactive', 'blocked'
  source_system       STRING,               -- Nguồn dữ liệu (CRM, Web, POS,...)
  ingestion_time      TIMESTAMP,            -- Thời điểm ingest
  ingestion_year      INT,                  -- Năm ingest
  ingestion_month     INT,                  -- Tháng ingest
  operation_type      STRING,               -- INSERT/UPDATE/DELETE
  record_hash         STRING,               -- Hash dùng kiểm tra thay đổi record
  source_updated_at   TIMESTAMP             -- Timestamp lần cuối cập nhật tại source
)
USING DELTA
PARTITIONED BY (ingestion_year, ingestion_month);
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = true,
  'comment' = 'Bảng raw chứa thông tin khách hàng từ hệ thống nguồn (ERP, CMS, POS)'
);

Tạo Comment cho các cột của bảng

In [0]:
%sql
COMMENT ON TABLE ecom_bronze.customer_raw IS 'Dữ liệu thô về khách hàng, ingest từ hệ thống CRM, web, POS,...';

COMMENT ON COLUMN ecom_bronze.customer_raw.customer_id IS 'Mã định danh duy nhất của khách hàng';
COMMENT ON COLUMN ecom_bronze.customer_raw.full_name IS 'Họ và tên khách hàng';
COMMENT ON COLUMN ecom_bronze.customer_raw.date_of_birth IS 'Ngày sinh khách hàng';
COMMENT ON COLUMN ecom_bronze.customer_raw.gender IS 'Giới tính: Nam, Nữ, Khác';
COMMENT ON COLUMN ecom_bronze.customer_raw.email IS 'Địa chỉ email khách hàng';
COMMENT ON COLUMN ecom_bronze.customer_raw.phone_number IS 'Số điện thoại';
COMMENT ON COLUMN ecom_bronze.customer_raw.address IS 'Địa chỉ cư trú';
COMMENT ON COLUMN ecom_bronze.customer_raw.city IS 'Thành phố khách hàng sinh sống';
COMMENT ON COLUMN ecom_bronze.customer_raw.country IS 'Quốc gia của khách hàng';
COMMENT ON COLUMN ecom_bronze.customer_raw.registration_date IS 'Thời điểm đăng ký tài khoản';
COMMENT ON COLUMN ecom_bronze.customer_raw.loyalty_level IS 'Cấp độ thành viên: Silver, Gold, Platinum';
COMMENT ON COLUMN ecom_bronze.customer_raw.status IS 'Trạng thái tài khoản: active, inactive, blocked';
COMMENT ON COLUMN ecom_bronze.customer_raw.source_system IS 'Hệ thống nguồn cung cấp dữ liệu';
COMMENT ON COLUMN ecom_bronze.customer_raw.ingestion_time IS 'Thời điểm ingest dữ liệu vào lakehouse';
COMMENT ON COLUMN ecom_bronze.customer_raw.ingestion_year IS 'Năm ingest dữ liệu';
COMMENT ON COLUMN ecom_bronze.customer_raw.ingestion_month IS 'Tháng ingest dữ liệu';
COMMENT ON COLUMN ecom_bronze.customer_raw.operation_type IS 'Loại thao tác trên dữ liệu: INSERT/UPDATE/DELETE';
COMMENT ON COLUMN ecom_bronze.customer_raw.record_hash IS 'Hash của bản ghi để kiểm tra thay đổi';
COMMENT ON COLUMN ecom_bronze.customer_raw.source_updated_at IS 'Thời điểm cập nhật cuối cùng ở nguồn';


Tạo các Constraint dữ liệu

In [0]:
%sql
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN customer_id SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN full_name SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN date_of_birth SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN gender SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN email SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN registration_date SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN loyalty_level SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN status SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN source_system SET NOT NULL;
ALTER TABLE ecom_bronze.customer_raw ALTER COLUMN ingestion_time SET NOT NULL;

-- Trạng thái hợp lệ
ALTER TABLE ecom_bronze.customer_raw 
ADD CONSTRAINT valid_status CHECK (status IN ('active', 'inactive', 'blocked'));

-- Hạng khách hàng hợp lệ
ALTER TABLE ecom_bronze.customer_raw 
ADD CONSTRAINT valid_loyalty CHECK (loyalty_level IN ('Silver', 'Gold', 'Platinum'));

ALTER TABLE ecom_bronze.customer_raw 
ADD CONSTRAINT gender_check CHECK (gender IN ('male', 'female'));

-- Không cho phép đăng ký sau thời điểm hiện tại
ALTER TABLE ecom_bronze.customer_raw 
ADD CONSTRAINT valid_registration_date CHECK (registration_date <= current_timestamp());

ALTER TABLE ecom_bronze.customer_raw 
ADD CONSTRAINT valid_ingestion_date CHECK (ingestion_time <= current_timestamp());

-- Không cho ngày sinh nhật sau thời điểm hiện tại
ALTER TABLE ecom_bronze.customer_raw 
ADD CONSTRAINT birthday_check CHECK (date_of_birth <= current_timestamp());

-- Kiểm tra email có dấu @ đơn giản
ALTER TABLE ecom_bronze.customer_raw 
ADD CONSTRAINT valid_email_format CHECK (email LIKE '%@%');
